# Cleanup the data and get the required field

In [2]:
sc.stop()

In [2]:
import sys
sc = SparkContext(appName='Taxi-Assignment')
lines = sc.textFile("./data/taxi-data-sorted-small.csv.bz2")
taxilines = lines.map(lambda x: x.split(','))

def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False
    
def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) != 0 and isfloat(p[11]) != 0):
                return p
            
textlinesCorrected = taxilines.filter(correctRows)

# Check if the dataset is cleanup.

In [4]:
textlinesCorrected.take(1)

[['07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  '2013-01-01 00:00:00',
  '2013-01-01 00:02:00',
  '120',
  '0.44',
  '-73.956528',
  '40.716976',
  '-73.962440',
  '40.715008',
  'CSH',
  '3.50',
  '0.50',
  '0.50',
  '0.00',
  '0.00',
  '4.50']]

In [15]:
type(textlinesCorrected)


pyspark.rdd.PipelinedRDD

In [16]:
taxi_driver = \
            textlinesCorrected.map(lambda x: (x[0], 1)).\
            reduceByKey(lambda x, y: x + y).top(10)

In [17]:
taxi_driver

[('FFFECF75AB6CC4FF9E8A8B633AB81C26', 240),
 ('FFF010F904EF7B60DAF12560AFE5127C', 1),
 ('FFEF5E9BA26FC9B6B5F2CF433833572D', 5),
 ('FFEE0D464EEA83AF6EA50CA5738B5610', 208),
 ('FFED7CFCB1A658ADEC2DC6A899047060', 309),
 ('FFE25FFFBDE6A6019A6A9051C4E798FA', 1),
 ('FFE1AB76511357473BE3236025321493', 171),
 ('FFD1575DA3F4DB85CCEC9FFBEAE16AEF', 291),
 ('FFCFA7AFF0DE2B5081C6C1A11099A691', 357),
 ('FFC6BE0FC070E78A65F9A0858D10ED63', 378)]

# 4.1 Task1: Top-10 Active Taxis

In [21]:
# The first map(taxi_driver_distinct) is to get the distinct relationship between taxi and driver.

taxi_driver_distinct = textlinesCorrected.map(lambda x : (x[0], x[1])).distinct()

# The next map is to separate taxi id from original rdd. The 'reducebykey' is to sum up 
# all the same taxis(which is a taxi map to the amount of drivers). Finally, I calculate 
# the top 10 taxis in pairs.

taxi_driver = taxi_driver_distinct.map(lambda x: (x[0], 1)).\
            reduceByKey(lambda x, y: x + y).top(10, lambda x: x[1])

taxi_driver


[('65EFB7D02BAD12D5DE757CB5D350944E', 20),
 ('3C08296D0EB7ABE24FB7328DE9B62813', 20),
 ('7DEB25123AE57111F912C0EBF92F1F63', 19),
 ('3B6AE3CF05F34ADC91DC68D20F2EB913', 19),
 ('F36564AB9C6EA3B6373EB0E1680A447A', 19),
 ('55D311AD2752BC278BEF7386B25B28A9', 19),
 ('CD7B02776E69483397952DC5E1F44DFE', 19),
 ('F2A08960199BCDB7EE19411A8E7A4C5D', 18),
 ('799153A138F4E8334A1A95AE25040B83', 18),
 ('9FB7A7C1D7B960D8B17829145C6A1CF2', 18)]

# 4.2 Task2: Top-10 Best Drivers

In [18]:
# First, I create a new rdd with total duration of the trip(in minutes) for each driver
driver_total_duration_min = textlinesCorrected.\
                            map(lambda x :(x[1], int(x[4])/60)).\
                            reduceByKey(lambda x,y: x+y)
driver_total_duration_min.top(10)

[('FFFBCEA3D4E21E05902EE67AD556F67C', 1779.0),
 ('FFF909B1353148850AD3E40BB878618B', 1528.0),
 ('FFF657CFEC6A06384C97ACB500916913', 1206.0),
 ('FFF6401CC16911710E7590FE197E986A', 370.0),
 ('FFF5AD65C673251C1F275CF5B43EC414', 8.45),
 ('FFF20BA1518E14B3B23F79DDDE1CA7E6', 2131.0),
 ('FFF0D1CE8F65C1A8B213128325EB5F33', 7.633333333333334),
 ('FFEE627F76A3A498DA53E200821FDEE9', 1965.0),
 ('FFEB80F39533ACF27F3124A39BB05E1E', 2395.0),
 ('FFDC72D8EE2737AEA6E0E9D7DDF4942E', 792.0)]

In [19]:
# Then, I create a new rdd with total amount of each driver got paid
driver_total_paid = textlinesCorrected.\
                    map(lambda x :(x[1], float(x[16]))).\
                    reduceByKey(lambda x,y: x+y)
driver_total_paid.top(10)

[('FFFBCEA3D4E21E05902EE67AD556F67C', 2364.69),
 ('FFF909B1353148850AD3E40BB878618B', 1785.4899999999998),
 ('FFF657CFEC6A06384C97ACB500916913', 1769.9300000000003),
 ('FFF6401CC16911710E7590FE197E986A', 466.75),
 ('FFF5AD65C673251C1F275CF5B43EC414', 17.77),
 ('FFF20BA1518E14B3B23F79DDDE1CA7E6', 2775.43),
 ('FFF0D1CE8F65C1A8B213128325EB5F33', 11.7),
 ('FFEE627F76A3A498DA53E200821FDEE9', 2916.2999999999997),
 ('FFEB80F39533ACF27F3124A39BB05E1E', 2883.8300000000004),
 ('FFDC72D8EE2737AEA6E0E9D7DDF4942E', 893.25)]

In [14]:
# The last step is to join these 2 rdd. Then calculate the best average 
best_average_earn = driver_total_duration_min.join(driver_total_paid).\
                    filter(lambda x: x[1][0] != 0).\
                    mapValues(lambda x: x[1]/x[0]).\
                    top(10, lambda x: x[1])
best_average_earn

[('30B2ACBAF002305533FF0D31D34A7C06', 702.0),
 ('4C3B2A31227663A59E1AA7B45157160D', 625.0),
 ('08026D69508127F4DE855678ABCE7E0A', 375.0),
 ('6E1D7195E38AA7A36B375C1C60AD8632', 317.3076923076923),
 ('E8E22AC46DF6AC99C28C6860779A648C', 273.0),
 ('975D5E840C0F5D9611A71E9A811D11F7', 190.90909090909093),
 ('17F72121B9F612D81AC4556CC18D5514', 107.14285714285714),
 ('619BF4020E6542AA0E28FBC7081271F1', 105.88235294117648),
 ('CD9D0B4429613F1B62D1D4FBB72ACF2B', 103.25172413793103),
 ('52C99F4F8CD2560F8793FE835B6B56BA', 89.28571428571429)]

In [1]:
sc.stop()

In [15]:
sc

<SparkContext master=local[*] appName=Taxi-Assignment>